# Linear regression using linreg

In [8]:
# create synthetic data and use linreg to compute slope and intercept
X = [1.422,4.123,8.734,11.223,14.659,18.266,20.456,25.123];
Y = [6.753,17.265,22.236,35.456,36.913,39.436,46.925,52.265];
print(typeof(X))
@time(linreg(X,Y))
a,b = linreg(X,Y);

Array{Float64,1}  0.132814 seconds (64.25 k allocations: 3.372 MiB)


In [16]:
# plot graph
using Gadfly
f(x) = a+b*x
#plotly() # Choose the Plotly.jl backend for web interactivity
plot(X,Y)
#plot!(f, 0, 30)

LoadError: [91mUndefVarError: plot not defined[39m

# Linear regression using ArrayFire

In [3]:
Pkg.add("ArrayFire")
using ArrayFire

INFO: Package ArrayFire is already installed


ArrayFire v3.5.1 (OpenCL, 64-bit Mac OSX, build 0a675e8)
[0] APPLE: AMD Radeon R9 M370X Compute Engine, 2048 MB
-1- APPLE: Iris Pro, 1536 MB


In [4]:
#ac = rand(10, 10)
#print(typeof(ac))
#ag = AFArray(ac)
#print(typeof(ag))
M = AFArray(X)
N = AFArray(Y)
@time(linreg(M,N))
a,b = linreg(M,N)
f(x) = a+b*x
plotly() # Choose the Plotly.jl backend for web interactivity
plot(X,Y,seriestype=:scatter,title="AF linear regression")
plot!(f, 0, 30)

  0.171345 seconds (70.37 k allocations: 3.817 MiB, 17.71% gc time)


In Julia, one would use the @time macro to time execution times of functions. However, in this particular case, @time would simply time the function call, and the library would execute asynchronously in the background. This would often lead to misleading timings. Therefore, the right way to time individual operations is to run them multiple times, place an explicit synchronization barrier at the end, and take the average of multiple runs.

# Linear Regression from scratch

In [11]:
using Flux.Tracker
using Plots


# random dataset
x, y = rand(5), rand(2)

# visualise dataset
plotly() # Choose the Plotly.jl backend for web interactivity
plot(x,y,seriestype=:scatter,title="linear regression")





LoadError: [91mUndefVarError: plot not defined[39m

In [ ]:
# randomly initiate weight and bias
W = param(rand(2, 5))
b = param(rand(2))

# using CuArrays for GPU support
# W, b, x, y = cu.((W, b, x, y))

predict(x) = W*x .+ b
meansquarederror(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)
loss(x, y) = meansquarederror(predict(x), y)

function update!(ps, η = .1)
  for w in ps
    #@show w.data
    #@show w.grad
    w.data .-= w.grad .* η  # apply update
    w.grad .= 0             # clear the gradient, why?
  end
end


for i = 1:5
  back!(loss(x, y))
  update!((W, b))
  @show loss(x, y)
end